In [1]:
import re
import math
import pickle
import random
import pandas as pd

from indic_transliteration import sanscript
from indic_transliteration.sanscript import SchemeMap, SCHEMES, transliterate


import time
import nltk
import numpy as np
import json
import copy
import sys
import codecs
from collections import *
from googletrans import Translator

In [2]:
'''These are the only 3 parameters that the user needs to declare in order to switch to a new language'''
language = 'Telugu'
script = 'TELUGU'
destCode = 'te'

if script == 'DEVANAGARI':
    trans =  sanscript.DEVANAGARI
elif script == 'TELUGU':
    trans =  sanscript.TELUGU



translator = Translator()

templateFile = language+'_Template.txt'
metaDataFile = language+'_meta.txt'

In [3]:
'''this part loads the template from the template and file creates the dictionary from the template '''

keyword_list = {}
word_list = defaultdict(list)

mapping_file_name=templateFile
words_map=open(mapping_file_name)
words=words_map.read()
words=words.split("\n")

key=""
value=[]
for word in words:
    if word == "" or word == " ":
        continue
    if "=" in word:
        if key != "":
            word_list[key]=value
        word = word.split("=")
        key = word[0]
        value = []
    else:
        value.append(word)
word_list[key] = value

print (word_list)

print (word_list['village_district_block'])

for key in word_list.keys():
    if key != 'Hospital_Name':
        print(word_list[key])

defaultdict(<class 'list'>, {'village_district_block': ['ఈ పాఠశాల $village గ్రామం లో ఉన్నది. ఈ గ్రామం $district జిల్లా లో $block పరిధి నందు ఉన్నది.', 'ఈ పాఠశాల $village గ్రామం లో ఉన్నది. $district జిల్లా నందు $block పరిధి లో ఈ గ్రామం ఉన్నది.', '$district జిల్లా లో $block పరిధి లో $village గ్రామం నందు ఈ పాఠశాల ఉన్నది. ', 'ఈ పాఠశాల $district జిల్లా లో $block పరిధి లో గల $village గ్రామము నందు ఉన్నది. '], 'CODE': ['ఈ పాఠశాల యొక్క ఏకీకృత జిల్లా సమాచార విద్యా వ్యవస్థ (U-DISE) కోడ్ $code '], 'BOTH_GENDER_CATEGORY': ['ఇది ఒక కో-ఎడ్యుకేషనల్ పాఠశాల.', 'ఇది ఒక బాల బాలికల పాఠశాల.'], 'BOYS_GENDER_CATEGORY': ['ఇది ఒక బాలుర పాఠశాల. ', 'ఈ పాఠశాల లో బాలురకు మాత్రమే ప్రవేశం ఉన్నది. ', 'బాలురకు మాత్రమే ఈ పాఠశాల నందు ప్రవేశం ఉన్నది.'], 'GIRLS_GENDER_CATEGORY': ['ఇది ఒక బాలికల పాఠశాల. ', 'ఈ పాఠశాల లో బాలికలకు మాత్రమే ప్రవేశం ఉన్నది.', 'బాలికలకు మాత్రమే ఇక్కడ ప్రవేశం ఉన్నది. '], 'BOTH_STUDENT_COUNT': ['ఇక్కడ $bcount బాలురు $gcount బాలికలు, మొత్తం $total విద్యార్థులు చదువుతున్నారు. ",', 'ఈ పాఠశాల లో మొత్తం $

In [4]:
'''Loads and process the metadata specific to a language '''

data_file_name=metaDataFile
meta_file=open(data_file_name)
lines=meta_file.read()
lines=lines.split("\n")
meta_list = defaultdict(list)

key=""
value=[]
for word in lines:
    if word == "" or word == " ":
        continue
    if "=" in word:
        if key != "":
            meta_list[key]=value
        word = word.split("=")
        key = word[0]
        value = []
    else:
        value.append(word)
meta_list[key] = value

print(meta_list)


letters = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']



metaL = meta_list['letters'][0].split()




dict_letters ={letters[i]:metaL[i] for i in range(len(letters))} 

#Special Lists
# sp_nums = ['', 'ఒక','ఇద్దరు','ముగ్గురు','నలుగురు','ఐదుగురు','ఆరుగురు','ఏడుగురు']
# sp_nums = ['', '','','','','','','']


class_nums = meta_list['class_nums'][0].split()



dictWord = {}
keylist = meta_list['sp_words_keys'][0].split()
valList = meta_list['sp_words_value'][0].split()
if (len(keylist)==len(valList)):
    for i in range(len(keylist)):
        dictWord[keylist[i]] = valList[i]


# print(len(telugu_nums))
# print(len(telugu_letters))

# print(telugu_nums[16])
# print(sp_nums[6])
# print(telugu_letters['a'])

defaultdict(<class 'list'>, {'letters': ['ఎ బి సి డి ఈ ఎఫ్ జి హెచ్ అయి జె కె ఎల్ ఎం ఎన్ ఓ పి క్యు అర్ ఎస్ టి యు వి డబల్యు ఎక్స్ వయి జెడ్'], 'class_nums': ["'' ఒకటవ రెండవ మూడవ నాలుగవ ఐదవ ఆరవ ఏడవ ఎనిమిదవ తొమ్మిదవ పదవ పదకొండవ పన్నెండవ"], 'sp_words_keys': ['anantapur guntur chitoor rural telugu english kannada'], 'sp_words_value': ['అనంతపూర్ గుంటూర్ చిత్తూరు రూరల్  లుగు ఇంగ్లీష్ కన్నడ']})


In [5]:
# dfile -> data from db
# pfile -> data from pdf


def getData(dfile,pfile):
    '''Loads the data from the files '''

    data = pickle.load(open(dfile,'rb'))
    df = pickle.load(open(pfile,'rb'))
    
    one =data['udise_sch_code'].to_list()
    two =df['Code'].to_list()
    two = [int(t) for t in two]
    
    intersection = sorted(set(one).intersection(set(two)))

    '''
    print("No. of codes in (1)DB - ", len(set(one)), " ,(2)PDF - ", len(set(two)))
    print("No. of codes NOT in PDF - ", len(set(one)-set(two)))
    print("No. of codes NOT in DB - ", len(set(two)-set(one)))
    print("No. of codes in BOTH - ", len(intersection))
    print("\n")
    '''
    return data, df, intersection

In [6]:
# సంబంధించినవి.
'''Trial and testing of translation and transliteration'''
data = 'guntur'
k = sanscript.DEVANAGARI
print(transliterate(data, sanscript.ITRANS, sanscript.TELUGU))
print(transliterate(data, sanscript.ITRANS, trans))
translator = Translator()
try:
    print (translator.translate('private aided', src='en', dest='hi').text)
except:
    pass


గున్తుర్
గున్తుర్


In [7]:

# Language specific code 

#case 0 (end) - pick ONLY from telugu_nums and NO మంది at the end
#case 1 (normal) - pick from telugu_nums and add మంది at the end or sp_nums 

def numToIndic(Int, case=1):
    '''Handles the numeric data '''
    Int = int(Int)
    
    # Flag to denote if picking from sp_nums
    sp = 0
    
    if Int > 100:
        num =str(Int)
        
    # elif 0<Int<8 and case==1:
        # sp = 1
        # num =sp_nums[Int]
        
    # elif Int<=20 or Int in [20,30,40,50,60,70,80,90,100]:
    #     num =hindi_nums[Int]
    
    # else:
    #     ones =Int%10
    #     tens = int((Int/10))*10
    #     num =hindi_nums[tens]+' '+hindi_nums[ones]
        
    # if case and not sp:
    #     # num+=' మంది'
    #     num += ''
        
    return transliterate(str(Int), sanscript.ITRANS, trans)+' '


def charsToIndic(abbr,f):

    tChars =''
    
    for c in abbr:
        if c.isalpha():
            c =dict_letters[c]+f
        tChars+=c
            
    return tChars

def preprocess(phrase):
    new = ''
    for c in phrase:
        if not c.isalpha() and c not in ['.','_']:
            c =' '+c+' '
        new+=c
    
    return new


def toIndic(phrase):
    tPhrase = ""
    
    phrase = phrase.lower()
    phrase = preprocess(phrase)
    
    words = phrase.split()
    
    spl = ['mpp','zpp','mpps','zphs','mpups','mjpapbcwrs','kgbv','aprei','nclp','icse','govt','cbse','dnt','apswrei','ssa','kgbvs']
    
    dot = '.'
    other = ''
    
    for word in words:
        tPhrase+=' '
        
        if word in dictWord.keys():
            tPhrase+=dictWord[word]
        
        elif word in spl or len(word)<3:
            tAbbr = charsToIndic(word, other)
            tPhrase+=tAbbr
        
        elif '.' in word or '_' in word:
            part =""
            for c in word:
                if c=='.':
                    if len(part)==1:
                        tPhrase+= charsToIndic(part,dot)
                    else:
                        tPhrase+= charsToIndic(part,other)+'.'
                    part =""
                elif c=='_':
                    tPhrase+=charsToIndic(part,other)+'_'
                    part =""
                elif c==' ':
                    if len(part)<3:
                        tPhrase+= charsToIndic(part,other)
                    else:
                        tPhrase+=transliterate(part, sanscript.ITRANS, trans)+' '
                    part = ""
                else:
                    part+=c
                    
            if part:
                if len(part)<3 or "_" in word:
                    tPhrase+= charsToIndic(part,other)
                else:
                    tPhrase+=transliterate(part, sanscript.ITRANS, trans)+' '
                    
        else:    
            tWord =transliterate(word, sanscript.ITRANS, trans) 
            tPhrase+=tWord+' '
    
#     print(phrase, tPhrase.strip())
#     print(tPhrase.strip())
    
    return tPhrase.strip()+' '


In [8]:
infoStr = '$label'
try:
    translator.translate('boys and girls', src='en', dest=destCode)
except:
    pass
# infoStr = infoStr.replace('$label',translator.translate('boys and girls', src='en', dest=destCode).text)


In [9]:
def getIntro(details,infoStr):
    code = str(details[3]).strip()
    name = toIndic(details[4].strip())
    village = toIndic(details[2].strip())
    district = toIndic(details[0].strip())
    block = toIndic(details[1].strip())

    infoStr = infoStr.replace('$name',name)
    infoStr = infoStr.replace('$district',district)
    infoStr = infoStr.replace('$village', village)
    

    introOpts = word_list['village_district_block']

    for i in range(len(introOpts)):
        introOpts[i] = introOpts[i].replace('$village',village)
        introOpts[i] = introOpts[i].replace('$district',district)
        introOpts[i] = introOpts[i].replace('$block',block)
                    
    
    intro = random.choice(introOpts)

    codeList = word_list['CODE']
    for i in range (len(codeList)):
        codeList[i]=codeList[i].replace('$code',code)

    intro = intro+random.choice(codeList)
    # print (intro)
    # print ()
    
    return intro,infoStr

# Returns Student's Type and Count of Students
def getStudentStats(counts,infoStr):
    stuType = counts[1].lower()
    bcount =numToIndic(counts[2])
    gcount = numToIndic(counts[3])
    total = numToIndic(int(counts[2])+int(counts[3]))
    
    sp=' '
    pick = 1
    
    infoStr = infoStr.replace('$totalStd',total)
    
    if 'co' in stuType:
        try:
            infoStr = infoStr.replace('$label',translator.translate('boys and girls', src='en', dest=destCode).text)
        except:
            pass


        opts1 = word_list['BOTH_GENDER_CATEGORY']

        opts2 = word_list['BOTH_STUDENT_COUNT']

        for i in range(len(opts2)):
            opts2[i] = opts2[i].replace('$bcount',bcount)
            opts2[i] = opts2[i].replace('$gcount',gcount)
            opts2[i] = opts2[i].replace('$total',total)
            

        
    elif 'boy' in stuType:
        try:
            infoStr = infoStr.replace('$label',translator.translate('boys', src='en', dest=destCode).text)
        except:
            pass

        opts1 = word_list['BOYS_GENDER_CATEGORY']

        opts2 = word_list['BOYS_STUDENT_COUNT']
        for i in range (len(opts2)):
            opts2[i]=opt2[i].replace('$bcount',bcount)
        
        
    elif 'girl' in stuType:

        try:
            infoStr = infoStr.replace('$label',translator.translate('girls', src='en', dest=destCode).text)
        except:
            pass


        opts1 =  word_list['GIRLS_GENDER_CATEGORY']

        opts2 = word_list['BOTH_STUDENT_COUNT']
        for i in range (len(opts2)):
            opts2[i]=opt2[i].replace('$Gcount',gcount)

    else:
        pick = 0
        
    if pick:
        line1 = random.choice(opts1)
        line2 = random.choice(opts2)
        studentStats =line1+line2
    else:
        studentStats = ''
        
    return studentStats,infoStr

#Returns Teacher's Stats and Reference
def getTeacherStats(counts,infoStr):
    sp = ' '
    mint = int(counts[5])
    fint =int(counts[6])
    mcount = numToIndic(mint)
    fcount = numToIndic(fint)
    total = numToIndic(mint+fint)
    infoStr = infoStr.replace('$totalFac', total)
    
    # if only male OR female teachers.
    if mint>0 and fint==0:
        lineList = word_list['MALE_TEACHER_COUNT']
        for i in range (len(lineList)):
            lineList[i] = lineList[i].replace('$mcount',mcount)
        line = random.choice(lineList)

    elif fint>0 and mint==0:
        lineList = word_list['FEMALE_TEACHER_COUNT']
        for i in range (len(lineList)):
            lineList[i] = lineList[i].replace('$fcount',fcount)
        line = random.choice(lineList)
        
    
    #Ekopadhya school
    elif mint==1 and fint==0:

        line = random.choice(word_list['ONE_TEACHER_MALE'])



    elif fint==1 and mint==0:
        line = random.choice(word_list['ONE_TEACHER_FEMALE'])


    
    else:
        total = numToIndic(mint+fint)


        opts = word_list['BOTH_TEACHER_COUNT']
        for i in range(len(opts)):
            opts[i] = opts[i].replace('$total',total)
            opts[i] = opts[i].replace('$mcount',mcount)
            opts[i] = opts[i].replace('$fcount',fcount)

        line = random.choice(opts)
            
    return line, infoStr
    
def getDesc(details,infoStr):
    name =details[4].strip()
    mgnt = details[6].strip()
    
    line = ''
    translator = Translator()
    if "Pvt.Aided" in mgnt:
        try:
            infoStr = infoStr.replace('$type',translator.translate('private aided', src='en', dest=destCode).text)
        except:
            pass
        line = ""
        line = random.choice(word_list['SCHOOL_CATEGORY_PVTAID'])

    elif "Pvt.Unaided" in mgnt:
        try:
            infoStr = infoStr.replace('$type',translator.translate('private unaided', src='en', dest=destCode).text)
        except:
            pass
        line = random.choice(word_list['SCHOOL_CATEGORY_PVT'])

    elif "KGBV" in mgnt:
        try:
            infoStr = infoStr.replace('$type',translator.translate('K.G.B.V', src='en', dest=destCode).text)
        except:
            pass        
        line = random.choice(word_list['SCHOOL_CATEGORY_KGBV'])
    
    elif "MPPS" in name:
        try:
            infoStr = infoStr.replace('$type',translator.translate('M.P.P.S', src='en', dest=destCode).text)
        except:
            pass
        line = random.choice(word_list['SCHOOL_CATEGORY_MPPS'])

    elif "MPPUS" in name:
        try:
            infoStr = infoStr.replace('$type',translator.translate('M.P.P.U.S', src='en', dest=destCode).text)
        except:
            pass
        line = random.choice(word_list['SCHOOL_CATEGORY_MPUPS'])

    elif "ZPPS" in name:
        try:
            infoStr = infoStr.replace('$type',translator.translate('Z.P.P.S', src='en', dest=destCode).text)
        except:
            pass
        line = random.choice(word_list['SCHOOL_CATEGORY_ZPPS'])

    elif "ZPHS" in name:
        try:
            infoStr = infoStr.replace('$type',translator.translate('Z.P.H.S', src='en', dest=destCode).text)
        except:
            pass
        line = random.choice(word_list['SCHOOL_CATEGORY_ZPHS'])
        
    return line, infoStr

def getInfo(data, df, Id):
    details = data.loc[data['udise_sch_code']==Id].values.tolist()[0]
    counts = df.loc[df['Code']==str(Id)].values.tolist()[0]
    
    return details, counts

def getClass(lo, hi,infoStr):
    if lo==1:
        lineList = word_list['CLASS_DISTINCTION_HI']
        for i in range (len(lineList)):
            lineList[i] = lineList[i].replace('$classHi',class_nums[hi])
        line = random.choice(lineList)


    else:
        lineList = word_list['CLASS_DISTINCTION']
        for i in range (len(lineList)):
            lineList[i] = lineList[i].replace('$classHi',class_nums[hi])
            lineList[i] = lineList[i].replace('$classLo',class_nums[lo])
        line = random.choice(lineList)
    
    infoStr = infoStr.replace('$classLo',class_nums[lo])
    infoStr = infoStr.replace('$classHi',class_nums[hi])

    return line, infoStr



def write(data, df, Id):
    infobox_name = 'infobox.txt'
    info_file=open(infobox_name)
    infoStr = info_file.read()

    article =""
    # Respective row details from pdf and excel data
    details, counts = getInfo(data, df, Id)
    
    #Sentence for name, village, district and block
    intro, infoStr = getIntro(details,infoStr)
    
    #Management
    management =toIndic(details[6].strip())

    opts = word_list['MANAGEMENT']
    for i in range (len(opts)):
        opts[i] = opts[i].replace('$management',management)


    descLine,infoStr =getDesc(details,infoStr)
    oLine =random.choice(opts)+descLine
    
    #Classes
    desc = details[5].strip()
    p = re.compile(r'[A-Z]+(\d+)-(\d+)')
    m =re.match(p, desc)
    lo =int(m.group(1)); hi =int(m.group(2))
    cLine,infoStr = getClass(lo, hi,infoStr)
    
    #School Medium
    medium = toIndic(details[7].strip())


    opts = word_list['MEDIUM']
    for i in range (len(opts)):
        opts[i] = opts[i].replace('$medium',medium)

    mLine = random.choice(opts)
    
    #School Type & student's counts
    studentStats,infoStr = getStudentStats(counts,infoStr)
    
    #Teachers counts + Reference Line
    teacherStats,infoStr = getTeacherStats(counts,infoStr)
    



    source = random.choice(word_list['SOURCE'])
    
    
    article =intro+oLine+cLine+mLine+studentStats+teacherStats+source
    print(infoStr)
    print(article)
    
    return article,infoStr


In [10]:
dfile_head = "DBpckls/"
pfile_head = "PDFpckls/"

dist = "ANANTAPUR"
data, df, Ids = getData(dfile_head+dist, pfile_head+dist)

In [11]:
#DEBUG TO GENERATE ONLY ONE ARTICLE
article,infoBox=write(data, df, 28222501207)

# print (infoBox.split())
for i in infoBox.split():
    if i[0]=='$':
        infoBox = infoBox.replace(i,' ')

print(infoBox+ article)
print(len(article.split()))

{{Infobox Secondary school
| name       = ఎంపియుపిఎస్ కె కె cఓలోన్య్  
| native_name   = ఎంపియుపిఎస్ కె కె cఓలోన్య్ 
| logo       = 
| motto      =
| established   =
| type       = $type
| category_label  =
| category     =
| gender_label   = विद्यार्थियों का लिंग
| gender      = boys and girls
| affiliations   =
| affiliation   =
| president    =
| chairman_label  =
| chairman     =
| principal    =
| asst principal  =
| vice principal  =
| campus director =
| headmaster    = 
| dean       =
| rector      =
| founder     =
| chaplain     =
| head_label    =
| head       =
| faculty     =
| teaching_staff  = ౧౧ 
| enrolement    =
| enrolment    =
| students     = ౩౦౦ 
| grades_label   = उम्र
| grades      = ఒకటవ-ఎనిమిదవ
| address     = 
| city       = కక్కలపల్లే  ( రూరల్ ) 
| state      = 
| province     =
| country     = 
| district     = అనంతపూర్ 
| oversight_label =
| oversight    =
| accreditation  =
| campus      =
| colors      =
| colours     = 
| athletics    =
| mascot      =


In [12]:
#GENERATING MULTIPLE ARTICLES
for dist in ["ANANTAPUR", "CHITTOOR","GUNTUR"]:
    print(dist)
    data, df, Ids = getData(dfile_head+dist, pfile_head+dist)

    failed = []
    for Id in Ids:
        try:
            article,infoBox =write(data, df, Id)
            print("pass")
        except:
            failed.append(Id)
            print('\t',Id)
        
    print('\t',failed)
    print('\t',len(failed))
    print('\t',len(Ids))

 లో అనంతపూర్  పరిధి లో గల కక్కలపల్లే  ( రూరల్ )  గ్రామము నందు ఉన్నది. ఈ పాఠశాల యొక్క ఏకీకృత జిల్లా సమాచార విద్యా వ్యవస్థ (U-DISE) కోడ్ 28222501207 ఎంపిపి_జెడ్పిపి స్చోఓల్స్  నిర్వహణ లో ఈ పాఠశాల ఉన్నది. ఇక్కడ విద్యార్థులు ఎనిమిదవ తరగతి వరకూ విద్యని అభ్యసించవచ్చు.ఈ పాఠశాల లో లుగు  మాధ్యమం లో పాఠాలు పాఠములు .ఇది ఒక కో-ఎడ్యుకేషనల్ పాఠశాల.ఈ పాఠశాల లో మొత్తం ౩౦౦  విద్యార్థులు చదువుతున్నారు. వీరిలో ౧౩౦  బాలురు ౧౭౦  బాలికలు ఉన్నారు. ఈ పాఠశాల లో మొత్తం ౧౧  ఉపాధ్యాయులు ఉన్నారు. వీరిలో ౪  ఉపాధ్యాయులు ౭  ఉపాధ్యాయినులు ఉన్నారు.ఈ వివరములు నేషనల్ యూనివర్శిటీ ఆఫ్ ఎడ్యుకేషనల్ ప్లానింగ్ అండ్ అడ్మినిస్ట్రేషన్ (NUEPA) జరిపిన సర్వే ఆధారముగా 2011-12 విద్యా సంవత్సరమునకు సంబంధించినవి
pass
{{Infobox Secondary school
| name       = ఎంపిపిఎస్ పులకుర్థి  
| native_name   = ఎంపిపిఎస్ పులకుర్థి 
| logo       = 
| motto      =
| established   =
| type       = $type
| category_label  =
| category     =
| gender_label   = विद्यार्थियों का लिंग
| gender      = boys and girls
| affiliations   =
| affiliation   =
| presi

In [ ]:
def main():
    dfile_head = "DBpckls/"
    pfile_head = "PDFpckls/"
    
    high=0
    big =''
    for dist in ["ANANTAPUR", "CHITTOOR", "GUNTUR"]:
        data, df, ids = getData(dfile_head+dist, pfile_head+dist)
        for Id in ids:
            atricle =write(data, df, Id)
